<a href="https://colab.research.google.com/github/Samir-atra/BDSD_Minor_Project/blob/main/MediaPipe_Stuff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download and install packages
# !python -m pip install mediapipe
# !wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task
# !wget -q -O image.png https://storage.googleapis.com/mediapipe-assets/business-person.png

# Imports
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt
import cv2
# from google.colab.patches import cv2_imshow
import csv
import pandas as pd
import tensorflow as tf
import keras_tuner
import math
import time

2024-04-22 01:23:11.469441: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-22 01:23:11.493721: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 01:23:11.493757: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 01:23:11.493775: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 01:23:11.498876: I tensorflow/core/platform/cpu_feature_g

Using TensorFlow backend


In [2]:
# MediaPipe visualizations


def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()


In [3]:
# a function to write to csv files
def csv_writer(filename, fields, data):

    csvfile = filename
    fields = fields
    with open(csvfile, mode="a") as first:
        csvwriter = csv.writer(first)
        csvwriter.writerow(fields)
        csvwriter.writerows(data)

        return True

In [4]:
# download and upload images
# download
# img = cv2.imread("images.jpg")
# cv2_imshow(img)

#upload
# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded:
#   content = uploaded[filename]
#   with open(filename, 'wb') as f:
#     f.write(content)

# if len(uploaded.keys()):
#   IMAGE_FILE = next(iter(uploaded))
#   print('Uploaded file:', IMAGE_FILE)

In [5]:
#inferencing and visualizing

# STEP 2: Create an FaceLandmarker object.
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file("image.png")

# STEP 4: Detect face landmarks from the input image.
detection_result = detector.detect(image)
# print(detection_result.facial_transformation_matrixes)

# STEP 5: Process the detection result. In this case, visualize it.
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
# img = plt.imshow(annotated_image)
# plt.show()
# cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

I0000 00:00:1713741793.221416   56751 task_runner.cc:85] GPU suport is not available: INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:84) egl_initializedUnable to initialize EGL
W0000 00:00:1713741793.221762   56751 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
libEGL 

In [6]:
# plot bar graphs

# plot_face_blendshapes_bar_graph(detection_result.face_blendshapes[0])
# print(detection_result.face_blendshapes[0])

In [7]:
# # Processing, cleaning and visualizing the dataset
# dataset = []
# Training_set = []
# validation_set = []
# test_set = []
# categories_count = {"0":0, "1":0, "2":0, "3":0, "4":0, "5":0, "6":0}
# skipped = {"0":0, "1":0, "2":0, "3":0, "4":0, "5":0, "6":0}

# with open("fer2013.csv", mode= "r") as data:
#   csvFile = csv.reader(data)
#   for lines in csvFile:
#       if lines[0] == "3":
#         lines[0] = "0"
#         dataset.append(lines)
#       elif lines[0] == "4":
#         lines[0] = '1'
#         dataset.append(lines)
#       elif lines[0] == '6':
#         lines[0] = '2'
#         dataset.append(lines) 
        
#   print(dataset[1:6])
  # # iterate over the training dataset
  # for i in range(len(Training_set)):
    # print(categories_count[Training_set[i][0]])
    # categories_count[Training_set[i][0]] = categories_count[Training_set[i][0]] + 1
    # image = np.array(Training_set[i][1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
    # image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    # print(image.shape)
    # rgb_frame = mp.Image(image_format=mp.ImageFormat.SRGB,data=image)
    # detection_result = detector.detect(rgb_frame)
    # print("emotion = ", Training_set[i][0])
    # img = plt.imshow(image)
    # plt.show()
    # # Skip and delete the image if not understood by mediapipe
    # if detection_result.face_blendshapes == []:
    #   skipped[Training_set[i][0]] = skipped[Training_set[i][0]] + 1
    # else:
    #   img = plt.imshow(image)
    #   plt.show()
  #     continue

  # print("This is categories count", categories_count)
  # print("This is skipped", skipped)


# plot_face_blendshapes_bar_graph(detection_result.face_blendshapes[0])
# annotated_image = draw_landmarks_on_image(rgb_frame.numpy_view(), detection_result)
# cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))



In [8]:
# cleaning the dataset

# training_set = []
# validation_set = []
# test_set = []

# for i in dataset:
#     if i[2] == 'Training':
#         training_set.append(i)
#     elif i[2] == 'PublicTest':
#         validation_set.append(i)
#     elif i[2] == 'PrivateTest':
#         test_set.append(i)

# training_set_hns = []
# validation_set_hns = []
# test_set_hns = []

# for lines in training_set:
#       image = np.array(lines[1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
#       image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
#     #   plt.imshow(image)
#     #   plt.show()
#       frame = mp.Image(image_format=mp.ImageFormat.SRGB,data=image)
#       detection_result = detector.detect(frame)
#       if detection_result.face_blendshapes == []:
#         continue
#       else:
#         training_set_hns.append(lines)

# for liness in validation_set:
#       imagee = np.array(liness[1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
#       imagee = cv2.cvtColor(imagee, cv2.COLOR_GRAY2RGB)
#     #   plt.imshow(image)
#     #   plt.show()
#       framee = mp.Image(image_format=mp.ImageFormat.SRGB,data=imagee)
#       detection_result = detector.detect(framee)
#       if detection_result.face_blendshapes == []:
#         continue
#       else:
#         validation_set_hns.append(liness)

# for linesss in test_set:
#       imageee = np.array(linesss[1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
#       imageee = cv2.cvtColor(imageee, cv2.COLOR_GRAY2RGB)
#     #   plt.imshow(image)
#     #   plt.show()
#       frameee = mp.Image(image_format=mp.ImageFormat.SRGB,data=imageee)
#       detection_result = detector.detect(frameee)
#       if detection_result.face_blendshapes == []:
#         continue
#       else:
#         test_set_hns.append(linesss)

# fields = ["emotion", "pixels", "Usage"]
# csv_writer("training_set_hns.csv", fields, training_set_hns)
# csv_writer("validation_set_hns.csv", fields, validation_set_hns)
# csv_writer("test_set_hns.csv", fields, test_set_hns)


In [9]:
# Augmenting images


# augmented_training_set = []
# training_images = []
# training_labels = []
# print(len(Training_set))

# for i in range(math.floor(len(training_set_hns))):
#     image = np.array(training_set_hns[i][1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
#     image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
#     training_images.append(image)
#     training_labels.append(int(training_set_hns[i][0]))


# # rescaling and augmenting images models
# rescaling1 = tf.keras.Sequential([
#   tf.keras.layers.Rescaling(1./255)
# ])

# rescaling2 = tf.keras.Sequential([
#   tf.keras.layers.Rescaling(1.*255)
# ])

# augment = tf.keras.Sequential([
#   tf.keras.layers.RandomFlip("horizontal"),
#   tf.keras.layers.RandomRotation(0.1)
# ])

# for ele in range(len(training_images)):
# scale down the image and augment
#   img = training_images[ele]
#   label = training_labels[ele]
#   image = rescaling1(img)
#   aug_image = augment(image)
# #   scale up the image cast to an integer and transform into a numpy array
#   aug_image = rescaling2(aug_image)
#   aug_image = tf.cast(aug_image, tf.uint8)
#   aug_image = np.array(aug_image)
#   flatten_image = aug_image.flatten()
#   flat_aug_image = [flatten_image[i] for i in range(0,len(flatten_image),3)]
#   # flattt = np.reshape(flat_aug_image,(48,48))
#   # plt.imshow(flattt)
#   # plt.show()
#   frame = mp.Image(image_format=mp.ImageFormat.SRGB,data=aug_image)
#   detection_result = detector.detect(frame)
#   if detection_result.face_blendshapes == []:
#     continue
#   else:
#     element = [training_labels[ele]]
#     for i in flat_aug_image:
#       element.append(i)
#     augmented_training_set.append([element[0],str(element[1:]).replace(',',"").replace('[','').replace(']',''),'Training'])

# csv_writer("training_set_hns.csv", ['emotion','pixels'], augmented_training_set)

# print(detection_result.face_blendshapes[0])
# annotated_image = draw_landmarks_on_image(frame.numpy_view(), detection_result)
# plt.imshow(annotated_image)
# plt.show()



In [10]:
# choosing relevant blendshapes
# blendS_to_print = ['1', '2', '3', '4', '5', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '25', '30', '31', '34', '35', '36', '37', '38', '40', '42', '44', '45', '46', '47', '48', '49', 'emotion']

# blend_shapes = dict()
# for i in range(0,52):
#     blend_shapes[str(i)] = 0

# print(blend_shapes)
# sad = 0
# happy = 0
# counter = 0

#   # find which blendshapes are most relevant to happiness and sadness
# for i in range(len(training_set_hns)):
#     image = np.array(training_set_hns[i][1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
#     image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
#     frame = mp.Image(image_format=mp.ImageFormat.SRGB,data=image)
#     detection_result = detector.detect(frame)
#     if detection_result.face_blendshapes == []:
#         continue
#     else:
#         counter += 1
#     if counter %500 == 0:
#         time.sleep(5)
#     for i in detection_result.face_blendshapes[0]:
#         if i.score > 0.4:
#             blend_shapes[str(i.index)] = blend_shapes[str(i.index)] + 1
# print(blend_shapes)

# detect annotate and plot blendshapes for the image
# frame = mp.Image(image_format=mp.ImageFormat.SRGB,data=aug_image)
# detection_result = detector.detect(frame)
# annotated_image = draw_landmarks_on_image(frame.numpy_view(), detection_result)
# cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
# plot_face_blendshapes_bar_graph(detection_result.face_blendshapes[0])

In [11]:
# if-else classification

# if detection_result.face_blendshapes[0][44].score > 0.5 and detection_result.face_blendshapes[0][45].score > 0.5 :
#   print("Happy")
# elif detection_result.face_blendshapes[0][42].score > 0.5:
#   print("Sad")


In [12]:
# creating blendshapes dataset

# val_set = []
# val_images = []
# val_labels = []

# blendS_to_print = ['1', '2', '3', '4', '5', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '25', '30', '31', '34', '35', '36', '37', '38', '40', '42', '44', '45', '46', '47', '48', '49','999']
# with open("validation_set_hns.csv", mode= "r") as data:
#   csvFile = csv.reader(data)
#   next(csvFile)
#   for lines in csvFile:
#     val_set.append(lines)
# #   print(test_set[0])
#   for i in range(len(val_set)):
#     # print(training_set[i][1])
#     image = np.array(val_set[i][1].split(' ')).reshape(48, 48, 1).astype(np.uint8)
#     image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
#     val_images.append(image)
#     val_labels.append(int(val_set[i][0]))

# arr = np.zeros((len(val_images), 34))

# print(len(val_images))
# arr[0,:]= blendS_to_print
# for ele in range(len(val_images)-1):
#   img = val_images[ele]
#   label = val_labels[ele]
#   frame = mp.Image(image_format=mp.ImageFormat.SRGB,data=img)
#   detection_result = detector.detect(frame)
# #   print(detection_result.face_blendshapes[0])
#   for category in detection_result.face_blendshapes[0]:
#     if str(category.index) in blendS_to_print:
#        arr[ele, blendS_to_print.index(f"{category.index}")] = category.score
#        arr[ele, 33] = label
#     else:
#        continue
    # print(arr)

# np.savetxt("blends_validation_set.csv", arr, delimiter = ",")

In [18]:
# LSTM model

blend_set = []
labels_set = []
with open("blends_train_set.csv", mode= "r") as data:
  csvFile = csv.reader(data)
  # next(csvFile)
  for lines in csvFile:
      blend_set.append(lines[0:33])
      labels_set.append(lines[33])
blends_set = np.array(blend_set, dtype=np.float64)
labels_set = np.array(labels_set, dtype=np.float64)
#Reshaping Array
X_train = np.reshape(blends_set, (30143, 33,1))
Y_train = np.reshape(labels_set, (30143,1)).astype('int')
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes=3)

# load validation data
val_blend_set = []
val_labels_set = []
with open("blends_validation_set.csv", mode= "r") as val_data:
  csvFile = csv.reader(val_data)
  # next(csvFile)
  for lines in csvFile:
      val_blend_set.append(lines[0:33])
      val_labels_set.append(lines[33])
val_blend_set = np.array(val_blend_set, dtype=np.float64)
val_labels_set = np.array(val_labels_set, dtype=np.float64)
X_val = np.reshape(val_blend_set, (1948, 33,1))
y_val = np.reshape(val_labels_set, (1948,1)).astype('int')
y_val = tf.keras.utils.to_categorical(y_val, num_classes=3)

# normalize data
# norm = tf.keras.layers.Normalization()
# norm.adapt(X_train)
# X_train = norm(X_train)
# X_train = np.array(X_train)

# norm.adapt(X_val)
# X_val = norm(X_val)
# X_val = np.array(X_val)

# def build_model(hp):

# model structure
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units = 33, activation = 'relu', return_sequences= True),
    tf.keras.layers.LSTM(units = 20, activation = 'relu', return_sequences= False),
    tf.keras.layers.Dense(units = 3, activation = 'softmax'),
])

  # learning_rate = hp.Float("lr", min_value=1e-6, max_value=1e-2, sampling="log")

# Compiling the RNN
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer= tf.keras.optimizers.Adam(learning_rate = 0.00464489028556945, clipnorm=1),
              metrics = [tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.F1Score()])

  # return model 

# build_model(keras_tuner.HyperParameters())

# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     max_trials=10,
#     objective='val_accuracy',
#     executions_per_trial=2,
#     overwrite=True,
#     directory="/home/samer/Desktop/Big data Small Data/BDSD/Minor_project/emotion_estimation/",
#     project_name="Emotion_estimation_tuning",
# )

# tuner.search_space_summary()

# tuner.search(x=X_train, y=Y_train, validation_data = (X_val,y_val), epochs=6)

# tuner.results_summary()



# EStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,patience=10, mode='auto',restore_best_weights=True)

# Fitting the RNN to the Training set
model.fit(x=X_train, y=Y_train, validation_data = (X_val,y_val) ,epochs = 100)


Epoch 1/100
942/942 [==============================] - 37s 38ms/step - loss: 0.7529 - categorical_accuracy: 0.6474 - f1_score: 0.5649 - val_loss: 0.7182 - val_categorical_accuracy: 0.6473 - val_f1_score: 0.5444
Epoch 2/100
200/942 [=====>........................] - ETA: 28s - loss: 0.6954 - categorical_accuracy: 0.6684 - f1_score: 0.6081

KeyboardInterrupt: 

In [17]:
# predict

test_blend_set = []
test_labels_set = []
with open("blends_test_set.csv", mode= "r") as test_data:
  csvFile = csv.reader(test_data)
  # next(csvFile)
  for lines in csvFile:
      test_blend_set.append(lines[0:33])
      test_labels_set.append(lines[33])
test_blend_set = np.array(test_blend_set, dtype=np.float64)
test_labels_set = np.array(test_labels_set, dtype=np.float64).astype('int')
test_labels_set = tf.keras.utils.to_categorical(test_labels_set, num_classes=3)
X_test = np.reshape(test_blend_set, (1915, 33,1))
model.evaluate(X_test,test_labels_set)


# model.save('LSTM_model_70%_train_acc')

60/60 [==============================] - 0s 8ms/step - loss: 1.7637 - categorical_accuracy: 0.2585 - f1_score: 0.1369
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1, 132), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f677bfcf550>, 140083080768128), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1, 132), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f677bfcf550>, 140083080768128), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(33, 132), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f677bf80220>, 140082911148144), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(33, 132), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f677bf80220>, 140082911148144), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(132,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f677bfa4c10>, 140083080788928), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(20, 3), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f677bf47d90>, 140083073833440), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f677bf55df0>, 140082910697712), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f677bf55df0>, 140082910697712), {}).


INFO:tensorflow:Assets written to: LSTM_model_70%_train_acc/assets


INFO:tensorflow:Assets written to: LSTM_model_70%_train_acc/assets
